In [1]:
import pandas as pd
from datetime import datetime
import re


In [38]:
rawpath = "../raw"

df = pd.read_excel(f"{rawpath}/Gesamt_Kbl2.xlsx", names=['signatur','urheber','idn','titel','zusatz','bem2','ort','verlag','entstehung','blatt','bild','reihe','preis','schlagwort','bemerkung','technik','motiv', 'idn_technik', 'mappe', 'grafiker'], dtype='string')
df = df.apply(lambda x: x.str.strip())
# filtere Tabelle in der Signatur nur Werte beginnend mit Kbl
df = df[(df.signatur.str.startswith('Kbl'))]
df = df.drop_duplicates(subset=['signatur'])
# droppe alle Zeilen, die einen Inhalt in Spalte Mappe haben
df[~pd.isna(df.mappe)].to_excel('mappenwerke.xlsx', index=False)
df = df[pd.isna(df.mappe)]
# lösche alle Zeilen bei Duplikaten in Spalte Sigantur


# speichere alle Zeilen mit Inhalten in Spalte Mappe in eine neue csv Datei
# df[~pd.isna(df.mappe)].to_excel('mappenwerke.xlsx', index=False)

In [39]:
# Suche in Spalte zusatz und erstelle neue Spalte hrsg
regsuche = re.compile(r"\[?[Hhrsg]{4}\.?\:?\]?(\s?[Vvonm]{1,3}\.?)?")
df['hrsg'] = df['zusatz'].str.split(regsuche, regex=True, n=1, expand=True)[2].str.strip()

In [40]:
# splite in spalte hrsg und füge 2. Teil in neue spalte hrsg2
df['hrsg2'] = df['hrsg'].str.split(',', expand=True)[1].str.strip()
# splite in spalte hrsg und überspeichere nur mit 1. Wert
df['hrsg'] = df['hrsg'].str.split(',', expand=True)[0].str.strip()

# ersetze regex-suche durch IDN in spalte hrsg2
regsuche =re.compile(r"Dt\.\sZentralinst\.?(itut)?\s[fF]\.\sLehrmittel")

df['hrsg2'] = df['hrsg2'].str.replace(regsuche, '!004686209!', regex=True)

# df['hrsg2'].dropna().to_csv('hrsg4.csv', index=False)


In [41]:
# ersetze regex-suche durch IDN in spalte hrsg
regsuche =re.compile(r"^Dt\.\s[pP]ädagog\.\sZentralinst.*")
df['hrsg'] = df['hrsg'].str.replace(regsuche, '!004686144!', regex=True)



In [6]:
def val_idn(idn):
    if len(idn) < 9:
        return (9 - len(idn)) * '0' + idn
    else:
        return idn

In [42]:
df[df["signatur"] == "Kbl 1964 C 8"]

,signatur,urheber,idn,titel,zusatz,bem2,ort,verlag,entstehung,blatt,...,preis,schlagwort,bemerkung,technik,motiv,idn_technik,mappe,grafiker,hrsg,hrsg2
1329,Kbl 1964 C 8,<NA>,<NA>,Hydraulischer Regler,"Hrsg. Vom Dt. Pädagog. Zentralinst., Dt. Zentr...",Mehrfarb. Offsetdruck,Berlin,Volk und Wissen,1964,1220 x 860,...,<NA>,Technik,<NA>,Offsetdruck,!041316169!,41333357,<NA>,Fritz Hampel,!004686144!,!004686209!


In [14]:

def pica_schreiben(row):
 
    pica = """0500 Pa
0501 unbewegtes Bild$bsti
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
"""
#4stellig suchen für 1100, 1110 der ganze Text der row.entstehung, falls nichts in row.entstehung steht dann folgender text 1100 1990$n[nicht nach 1990] re.search r"1\d{3}"
    if  (pd.notna(row.entstehung)) and (year := re.findall(r"\d{4}", str(row.entstehung))):
        pica += f"1100 {year[0]}\n1110 {row.entstehung}$4ezth\n"
    else:
        pica += f"1100 1991$n[nicht nach 1991]\n"

    pica +=f"1131 !041512367!\n"
    if 'year' in locals():
        if int(year[0]) > 1944:
            pica +=f"1700 /1XA-DDDE\n"
        elif int(year[0]) < 1945:
            pica +=f"1700 /1XA-DXDE\n"

    pica +=f"2105 94,P01\n"

    if pd.notna(row.preis):
        pica +=f"2000 :{row.preis}\n"

    if pd.notna(row.idn):
        pica +=f"3010 !{val_idn(row.idn)}!$BKünstler$4art\n" 
    elif pd.notna(row.urheber):
        pica +=f"3010 {row.urheber}$BKünstler$4art\n"

# splite grafiker am Leerzeichen und füge zuerst 2.Wert, 1. 1.Wert ein

# row.urheber.replace("[","").replace("]","").split(", ")

    if pd.notna(row.grafiker):
        grafiker_split = row.grafiker.split(" ")
        if len(grafiker_split) > 1:    
            pica += f"3010 {grafiker_split[1]}, {grafiker_split[0]}$BKünstler$4art\n"
    

    if pd.notna(row.hrsg):
        pica +=f"3100 {row.hrsg}$BHerausgebendes Organ$4isb\n"
    if pd.notna(row.hrsg2) and (row.hrsg2 == "!004686209!"):
        pica +=f"3110 {row.hrsg2}$BHerausgebendes Organ$4isb\n"

    if pd.notna(row.titel):
        pica +=f"4000 {row.titel}"
    
    if pd.notna(row.zusatz):
        pica += f" : {row.zusatz}"

    if pd.notna(row.urheber) or pd.notna(row.grafiker):
        pica += " / "

    if pd.notna(row.urheber):
        urheber = row.urheber.split(", ")

        
        if len(urheber) > 1:
            pica += f"{urheber[1]} "

        pica += f"{urheber[0]}"

    if pd.notna(row.grafiker):
        pica += f"Grafiker: {row.grafiker}"

    pica += "\n"

    if pd.notna(row.ort) and pd.notna(row.verlag):
        pica +=f"4030 {row.ort} : {row.verlag}\n"
    elif pd.isna(row.ort) and pd.notna(row.verlag):
        pica +=f"4030 [Ort nicht ermittelbar] : {row.verlag}\n"
    elif pd.notna(row.ort) and pd.isna(row.verlag):
        pica +=f"4030 {row.ort} : [Verlag nicht ermittelbar]\n"
    elif pd.isna(row.ort) and pd.isna(row.verlag):
        pica +=f"4030 [Ort nicht ermittelbar] : [Verlag nicht ermittelbar]\n"

    pica +=f"4060 1 ungezähltes Blatt\n"

    if blattwerte := re.findall(r"(\d+)\s?x\s?(\d+)", str(row.blatt)):
        pica += f"4062 {blattwerte[0][0]} x {blattwerte[0][1]} mm$b{blattwerte[0][0]}$h{blattwerte[0][1]}$4bltt\n"

    if bildwerte := re.findall(r"(\d+)\s?x\s?(\d+)", str(row.bild)):
        pica += f"4062 {bildwerte[0][0]} x {bildwerte[0][1]} mm$b{bildwerte[0][0]}$h{bildwerte[0][1]}$4bild\n"

    # pica +=f"4105 !!\n"



    if pd.notna(row.bem2) and pd.notna(row.bemerkung):
        pica +=f"4201 {row.bem2}. {row.bemerkung}\n"

    elif pd.isna(row.bem2) and pd.notna(row.bemerkung):
        pica +=f"4201 {row.bemerkung}\n"

    elif pd.notna(row.bem2) and pd.isna(row.bemerkung):
        pica +=f"4201 {row.bem2}\n"

    if pd.notna(row.reihe):
        pica +=f"4190 {row.reihe}\n"

    if pd.notna(row.motiv):
        pica +=f"5100 {row.motiv}\n"

    if pd.notna(row.idn_technik):
        pica +=f"5590 [Technik]\n5590 !{val_idn(row.idn_technik)}!\n"

    pica +=f"""7001 x
4800 !009033645!
7100 {row.signatur} @ m
7109 !!DBSM/S!! ; {row.signatur}
"""


        



    pica += "\t\n"
    return pica

In [8]:
for index, row in df.sample(10).iterrows():
    print(pica_schreiben(row))



0500 Pa
0501 unbewegtes Bild$bsti
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1959
1110 1959$4ezth
1131 !041512367!
1700 /1XA-DDDE
2105 94,P01
2000 :19
3010 !119425157!$BKünstler$4art
4000 Flusslandschaft / Paul Signac
4030 Dresden : Verl. d. Kunst
4060 1 ungezähltes Blatt
4062 412 x 512 mm$b412$h512$4bltt
4062 295 x 415 mm$b295$h415$4bild
4201 Mehrfarb. Lichtdruck
4190 Dresdner Meisterdruck. M 1157
5590 [Technik]
5590 !042736064!
7001 x
4800 !009033645!
7100 Kbl 1959 C 11 @ m
7109 !!DBSM/S!! ; Kbl 1959 C 11
	

0500 Pa
0501 unbewegtes Bild$bsti
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1967
1110 1967$4ezth
1131 !041512367!
1700 /1XA-DDDE
2105 94,P01
2000 :4,50 DM (DDR)
3010 !141446897!$BKünstler$4art
4000 Aschenputtel / Siegfried Linke
4030 Reichenbach i. V. : Bild und Heimat
4060 1 ungezähltes Blatt
4062 350 x 170 mm$b350$h170$4bltt
4201 Mehrfarb. Offsetdruck
5100 !041319869!
5590 [Technik]
5590 !041333357!
7001 x
4800 !009033645!
7100 Kbl 1

In [15]:
now = datetime.now()

with open(f"../dat/kunstblatt-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/kunstblatt-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))

In [ ]:
df.to_csv('daten.csv', index=False)

: 